LlamaIndex是一个功能强大的框架，让开发者能够轻松构建与自定义数据交互的LLM应用。LlamaIndex数据处理的核心工作流程：加载数据、建立索引、查询响应。在今天的学习中，我们将深入探讨每个步骤的更多细节和高级用法。你将掌握以下使用LlamaIndex构建RAG系统的基础知识：

数据加载：从多种源加载文档
文本切分：将文档分割成适合检索的小块
索引构建：使用向量存储等技术创建检索索引
检索实现：基于相似度等方法检索相关内容
回答生成：将检索内容与用户问题结合生成回答
高级定制：自定义模型、提示词和处理流程

## 1. 数据加载

LlamaIndex提供了丰富的数据加载工具，可以从各种来源导入数据。

### 1.1 基础文件加载

最常用的文档加载器是SimpleDirectoryReader，它可以加载指定目录下的各种格式文件：

In [1]:
from llama_index.core import SimpleDirectoryReader

# 基本使用
reader = SimpleDirectoryReader(
    input_dir="./data/llamaindex_data/",  # 目标目录
    recursive=False,     # 是否递归遍历子目录
    required_exts=[".pdf", ".txt"]  # 只读取指定后缀的文件
)
documents = reader.load_data()

每个文档都包含文本内容和元数据，可以通过以下方式查看：

In [2]:
# 查看文档内容
print(documents[0].text)

# 查看文档完整信息（包括元数据）
import json
print(json.dumps(documents[0].dict(), indent=4, ensure_ascii=False))

DeepSeek-V3 Technical Report
DeepSeek-AI
research@deepseek.com
Abstract
We present DeepSeek-V3, a strong Mixture-of-Experts (MoE) language model with 671B total
parameters with 37B activated for each token. To achieve efficient inference and cost-effective
training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architec-
tures, which were thoroughly validated in DeepSeek-V2. Furthermore, DeepSeek-V3 pioneers
an auxiliary-loss-free strategy for load balancing and sets a multi-token prediction training
objective for stronger performance. We pre-train DeepSeek-V3 on 14.8 trillion diverse and
high-quality tokens, followed by Supervised Fine-Tuning and Reinforcement Learning stages to
fully harness its capabilities. Comprehensive evaluations reveal that DeepSeek-V3 outperforms
other open-source models and achieves performance comparable to leading closed-source
models. Despite its excellent performance, DeepSeek-V3 requires only 2.788M H800 GPU hours
for its full trai

### 3.2 使用LlamaParse处理PDF文档

对于PDF文件，LlamaIndex提供了更专业的解析器LlamaParse，可以更好地保留文档的结构：

In [4]:
from llama_cloud_services import LlamaParse
from llama_index.core import SimpleDirectoryReader
import nest_asyncio
import os

# 在Jupyter环境中需要此操作
nest_asyncio.apply()

# 设置解析器
parser = LlamaParse(
    result_type="markdown"  # 可选"markdown"或"text"
)
file_extractor = {".pdf": parser}

documents = SimpleDirectoryReader(
    input_dir="./data/llamaindex_data/", 
    required_exts=[".pdf"], 
    file_extractor=file_extractor
).load_data()

print(documents[0].text)

Started parsing the file under job_id dbc57e94-2001-421f-903e-9f131d863c54
# DeepSeek-V3 Technical Report

# DeepSeek-AI

research@deepseek.com

# Abstract

We present DeepSeek-V3, a strong Mixture-of-Experts (MoE) language model with 671B total parameters with 37B activated for each token. To achieve efficient inference and cost-effective training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architectures, which were thoroughly validated in DeepSeek-V2. Furthermore, DeepSeek-V3 pioneers an auxiliary-loss-free strategy for load balancing and sets a multi-token prediction training objective for stronger performance. We pre-train DeepSeek-V3 on 14.8 trillion diverse and high-quality tokens, followed by Supervised Fine-Tuning and Reinforcement Learning stages to fully harness its capabilities. Comprehensive evaluations reveal that DeepSeek-V3 outperforms other open-source models and achieves performance comparable to leading closed-source models. Despite its excel

### 3.3 加载网页数据

LlamaIndex还支持直接从网页加载数据：

In [14]:
# 安装网页数据解析器
# %pip install llama-index-readers-web

from llama_index.readers.web import SimpleWebPageReader

# 加载网页数据
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://qwenlm.github.io/zh/blog/qwen3/", ]
)

print(documents[0].text)

[![](https://qwenlm.github.io/img/logo.png)](/ "Qwen \(Alt + H\)")

  * [Blog](/blog/ "Blog")
  * [Publication](/publication "Publication")
  * [About](/about "About")
  * [Try Qwen Chat ](https://chat.qwen.ai "Try Qwen Chat")

# Qwen3：思深，行速

2025年4月29日 · 4 分钟 · 794 字 · Qwen Team | 语言:

  * [English](https://qwenlm.github.io/blog/qwen3/)

![Qwen3 Main Image](https://qianwen-res.oss-accelerate-
overseas.aliyuncs.com/qwen3-banner.png)

[QWEN CHAT](https://chat.qwen.ai) [GitHub](https://github.com/QwenLM/Qwen3)
[Hugging
Face](https://huggingface.co/collections/Qwen/qwen3-67dd247413f0e2e4f653967f)
[ModelScope](https://modelscope.cn/collections/Qwen3-9743180bdc6b48)
[Kaggle](https://www.kaggle.com/models/qwen-lm/qwen-3)
[DEMO](https://huggingface.co/spaces/Qwen/Qwen3-Demo)
[DISCORD](https://discord.gg/yPEP2vHTu4)

## 引言#

今天，我们宣布推出 **Qwen3** ，这是 Qwen 系列大型语言模型的最新成员。我们的旗舰模型 **Qwen3-235B-A22B**
在代码、数学、通用能力等基准测试中，与 DeepSeek-R1、o1、o3-mini、Grok-3 和 Gemini-2.5-Pro
等顶级模型相比，表现出极具竞争力的结果。此外，小型 MoE 模型 

## 2. 文本切分与解析

为了便于检索，LlamaIndex将文档切分为更小的文本片段（Node），这个过程称为Chunking。

### 2.1 使用TokenTextSplitter切分文本

In [15]:
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter

# 创建文本分割器
node_parser = TokenTextSplitter(
    chunk_size=1000,    # 每个chunk的最大长度
    chunk_overlap=500   # chunk之间重叠长度
)

# 切分文档
nodes = node_parser.get_nodes_from_documents(documents)

# 查看节点内容
print(nodes[0].text)

[![](https://qwenlm.github.io/img/logo.png)](/ "Qwen \(Alt + H\)")

  * [Blog](/blog/ "Blog")
  * [Publication](/publication "Publication")
  * [About](/about "About")
  * [Try Qwen Chat ](https://chat.qwen.ai "Try Qwen Chat")

# Qwen3：思深，行速

2025年4月29日 · 4 分钟 · 794 字 · Qwen Team | 语言:

  * [English](https://qwenlm.github.io/blog/qwen3/)

![Qwen3 Main Image](https://qianwen-res.oss-accelerate-
overseas.aliyuncs.com/qwen3-banner.png)

[QWEN CHAT](https://chat.qwen.ai) [GitHub](https://github.com/QwenLM/Qwen3)
[Hugging
Face](https://huggingface.co/collections/Qwen/qwen3-67dd247413f0e2e4f653967f)
[ModelScope](https://modelscope.cn/collections/Qwen3-9743180bdc6b48)
[Kaggle](https://www.kaggle.com/models/qwen-lm/qwen-3)
[DEMO](https://huggingface.co/spaces/Qwen/Qwen3-Demo)
[DISCORD](https://discord.gg/yPEP2vHTu4)

## 引言#

今天，我们宣布推出 **Qwen3** ，这是 Qwen 系列大型语言模型的最新成员。我们的旗舰模型 **Qwen3-235B-A22B**
在代码、数学、通用能力等基准测试中，与 DeepSeek-R1、o1、o3-mini、Grok-3 和 Gemini-2.5-Pro
等顶级模型相比，表现出极具竞争力的结果。此外，小型 MoE 模型 

LlamaIndex提供了多种文本分割器，满足不同场景需求：

SentenceSplitter：在切分时尽量保证句子边界不被切断

CodeSplitter：根据AST（抽象语法树）切分代码，保证代码功能片段完整

SemanticSplitterNodeParser：根据语义相关性切分文本

### 2.2 使用NodeParsers解析结构化文档

对于HTML等结构化文档，可以使用专门的解析器提取有意义的内容：

In [24]:
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.readers.web import SimpleWebPageReader

# 加载HTML文档
documents = SimpleWebPageReader(html_to_text=False).load_data(["https://qwenlm.github.io/zh/blog/qwen3/"])

# 解析HTML标签
# 默认解析 ["p", "h1", "h2", "h3", "h4", "h5", "h6", "li", "b", "i", "u", "section"]
parser = HTMLNodeParser(tags=["p"])  # 可以自定义解析哪些标签
nodes = parser.get_nodes_from_documents(documents)

for node in nodes:
    print(node.text + "\n")

QWEN CHAT
GitHub
Hugging Face
ModelScope
Kaggle
DEMO
DISCORD
今天，我们宣布推出
Qwen3
，这是 Qwen 系列大型语言模型的最新成员。我们的旗舰模型
Qwen3-235B-A22B
在代码、数学、通用能力等基准测试中，与 DeepSeek-R1、o1、o3-mini、Grok-3 和 Gemini-2.5-Pro 等顶级模型相比，表现出极具竞争力的结果。此外，小型 MoE 模型
Qwen3-30B-A3B
的激活参数数量是 QwQ-32B 的 10%，表现更胜一筹，甚至像 Qwen3-4B 这样的小模型也能匹敌 Qwen2.5-72B-Instruct 的性能。
我们开源了两个 MoE 模型的权重：
Qwen3-235B-A22B
，一个拥有 2350 多亿总参数和 220 多亿激活参数的大模型，以及
Qwen3-30B-A3B
，一个拥有约 300 亿总参数和 30 亿激活参数的小型 MoE 模型。此外，六个 Dense 模型也已开源，包括
Qwen3-32B
、
Qwen3-14B
、
Qwen3-8B
、
Qwen3-4B
、
Qwen3-1.7B
和
Qwen3-0.6B
，均在 Apache 2.0 许可下开源。
经过后训练的模型，例如
Qwen3-30B-A3B
，以及它们的预训练基座模型（如
Qwen3-30B-A3B-Base
），现已在
Hugging Face
、
ModelScope
和
Kaggle
等平台上开放使用。对于部署，我们推荐使用
SGLang
和
vLLM
等框架；而对于本地使用，像
Ollama
、
LMStudio
、
MLX
、
llama.cpp
和
KTransformers
这样的工具也非常值得推荐。这些选项确保用户可以轻松将 Qwen3 集成到他们的工作流程中，无论是用于研究、开发还是生产环境。
我们相信，Qwen3 的发布和开源将极大地推动大型基础模型的研究与开发。我们的目标是为全球的研究人员、开发者和组织赋能，帮助他们利用这些前沿模型构建创新解决方案。
欢迎在 Qwen Chat 网页版 (
chat.qwen.ai
) 和手机 APP 中试用 Qwen3！

Qwen3 模型支持两种思考模式：
这种灵活性使用户能够根据具体任务控制模型进行“思考”

## 3. 索引与检索
索引是为实现快速检索而设计的特定数据结构。LlamaIndex提供多种索引类型，最常用的是向量索引。

### 3.1 向量检索
VectorStoreIndex是最基础的索引类型，将文本转换为向量并存储在内存中：

In [25]:
from llama_index.core import VectorStoreIndex

# 构建索引
index = VectorStoreIndex(nodes)

# 创建检索器
retriever = index.as_retriever(
    similarity_top_k=3  # 返回最相似的3个结果
)

# 检索
results = retriever.retrieve("qwen3是什么?")

# 查看检索结果
for result in results:
    print(f"相关度分数: {result.score}")
    print(result.text)
    print("-" * 50)

相关度分数: 0.8638171847866085
QWEN CHAT
GitHub
Hugging Face
ModelScope
Kaggle
DEMO
DISCORD
今天，我们宣布推出
Qwen3
，这是 Qwen 系列大型语言模型的最新成员。我们的旗舰模型
Qwen3-235B-A22B
在代码、数学、通用能力等基准测试中，与 DeepSeek-R1、o1、o3-mini、Grok-3 和 Gemini-2.5-Pro 等顶级模型相比，表现出极具竞争力的结果。此外，小型 MoE 模型
Qwen3-30B-A3B
的激活参数数量是 QwQ-32B 的 10%，表现更胜一筹，甚至像 Qwen3-4B 这样的小模型也能匹敌 Qwen2.5-72B-Instruct 的性能。
我们开源了两个 MoE 模型的权重：
Qwen3-235B-A22B
，一个拥有 2350 多亿总参数和 220 多亿激活参数的大模型，以及
Qwen3-30B-A3B
，一个拥有约 300 亿总参数和 30 亿激活参数的小型 MoE 模型。此外，六个 Dense 模型也已开源，包括
Qwen3-32B
、
Qwen3-14B
、
Qwen3-8B
、
Qwen3-4B
、
Qwen3-1.7B
和
Qwen3-0.6B
，均在 Apache 2.0 许可下开源。
经过后训练的模型，例如
Qwen3-30B-A3B
，以及它们的预训练基座模型（如
Qwen3-30B-A3B-Base
），现已在
Hugging Face
、
ModelScope
和
Kaggle
等平台上开放使用。对于部署，我们推荐使用
SGLang
和
vLLM
等框架；而对于本地使用，像
Ollama
、
LMStudio
、
MLX
、
llama.cpp
和
KTransformers
这样的工具也非常值得推荐。这些选项确保用户可以轻松将 Qwen3 集成到他们的工作流程中，无论是用于研究、开发还是生产环境。
我们相信，Qwen3 的发布和开源将极大地推动大型基础模型的研究与开发。我们的目标是为全球的研究人员、开发者和组织赋能，帮助他们利用这些前沿模型构建创新解决方案。
欢迎在 Qwen Chat 网页版 (
chat.qwen.ai
) 和手机 APP 中试用 Qwen3！

Qwen3 模型支持两种思考模式：


默认情况下，LlamaIndex使用OpenAI的Embedding模型，但你可以轻松替换为其他模型。

### 3.2 使用外部向量数据库

在实际应用中，通常需要使用专门的向量数据库存储向量以提高性能和持久化数据。以Qdrant为例：

In [28]:
# 安装Qdrant扩展
# %pip install llama-index-vector-stores-qdrant

from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# 创建Qdrant客户端
client = QdrantClient(location=":memory:")  # 生产环境可使用真实服务器地址
collection_name = "my_collection"
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

# 创建向量存储和索引
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes, storage_context=storage_context)

# 检索
retriever = index.as_retriever(similarity_top_k=3)
results = retriever.retrieve("qwen3是什么?")

print(results[0].text)

QWEN CHAT
GitHub
Hugging Face
ModelScope
Kaggle
DEMO
DISCORD
今天，我们宣布推出
Qwen3
，这是 Qwen 系列大型语言模型的最新成员。我们的旗舰模型
Qwen3-235B-A22B
在代码、数学、通用能力等基准测试中，与 DeepSeek-R1、o1、o3-mini、Grok-3 和 Gemini-2.5-Pro 等顶级模型相比，表现出极具竞争力的结果。此外，小型 MoE 模型
Qwen3-30B-A3B
的激活参数数量是 QwQ-32B 的 10%，表现更胜一筹，甚至像 Qwen3-4B 这样的小模型也能匹敌 Qwen2.5-72B-Instruct 的性能。
我们开源了两个 MoE 模型的权重：
Qwen3-235B-A22B
，一个拥有 2350 多亿总参数和 220 多亿激活参数的大模型，以及
Qwen3-30B-A3B
，一个拥有约 300 亿总参数和 30 亿激活参数的小型 MoE 模型。此外，六个 Dense 模型也已开源，包括
Qwen3-32B
、
Qwen3-14B
、
Qwen3-8B
、
Qwen3-4B
、
Qwen3-1.7B
和
Qwen3-0.6B
，均在 Apache 2.0 许可下开源。
经过后训练的模型，例如
Qwen3-30B-A3B
，以及它们的预训练基座模型（如
Qwen3-30B-A3B-Base
），现已在
Hugging Face
、
ModelScope
和
Kaggle
等平台上开放使用。对于部署，我们推荐使用
SGLang
和
vLLM
等框架；而对于本地使用，像
Ollama
、
LMStudio
、
MLX
、
llama.cpp
和
KTransformers
这样的工具也非常值得推荐。这些选项确保用户可以轻松将 Qwen3 集成到他们的工作流程中，无论是用于研究、开发还是生产环境。
我们相信，Qwen3 的发布和开源将极大地推动大型基础模型的研究与开发。我们的目标是为全球的研究人员、开发者和组织赋能，帮助他们利用这些前沿模型构建创新解决方案。
欢迎在 Qwen Chat 网页版 (
chat.qwen.ai
) 和手机 APP 中试用 Qwen3！

Qwen3 模型支持两种思考模式：
这种灵活性使用户能够根据具体任务控制模型进行“思考”

### 3.3 检索后处理

LlamaIndex提供了检索后处理模块，可以进一步优化检索结果。例如，使用语言模型对检索结果重新排序：

In [30]:
from llama_index.core.postprocessor import LLMRerank

# 创建检索器并获取结果
retriever = index.as_retriever(similarity_top_k=5)
nodes = retriever.retrieve("qwen3是什么?")

# 使用LLM重排序，保留最相关的前2个结果
postprocessor = LLMRerank(top_n=2)
reranked_nodes = postprocessor.postprocess_nodes(
    nodes, 
    query_str="qwen3是什么?"
)

# 查看重排序后的结果
for i, node in enumerate(reranked_nodes):
    print(f"[{i}] {node.text[:300]}...")

[0] QWEN CHAT
GitHub
Hugging Face
ModelScope
Kaggle
DEMO
DISCORD
今天，我们宣布推出
Qwen3
，这是 Qwen 系列大型语言模型的最新成员。我们的旗舰模型
Qwen3-235B-A22B
在代码、数学、通用能力等基准测试中，与 DeepSeek-R1、o1、o3-mini、Grok-3 和 Gemini-2.5-Pro 等顶级模型相比，表现出极具竞争力的结果。此外，小型 MoE 模型
Qwen3-30B-A3B
的激活参数数量是 QwQ-32B 的 10%，表现更胜一筹，甚至像 Qwen3-4B 这样的小模型也能匹敌 Qwen2.5-...


## 4. 生成回复

LlamaIndex提供两种主要的响应接口：QueryEngine（单轮问答）和ChatEngine（多轮对话）。

### 4.1 单轮问答（QueryEngine）

In [31]:
# 创建问答引擎
qa_engine = index.as_query_engine()

# 提问
response = qa_engine.query("qwen3是什么?")
print(response)

# 流式输出
streaming_qa_engine = index.as_query_engine(streaming=True)
response = streaming_qa_engine.query("qwen3是什么?")
response.print_response_stream()

Qwen3 是一系列大型语言模型中的最新成员，包括 Qwen3-235B-A22B 和 Qwen3-30B-A3B 等模型。这些模型在代码、数学、通用能力等基准测试中表现出极具竞争力的结果，并已经在多个平台上开放使用。其支持两种思考模式，拥有广泛的多语言能力，以及经过优化的 Agent 和代码能力，同时加强了对 MCP 的支持。
Qwen3 是一系列大型语言模型中的最新成员，包括 Qwen3-235B-A22B 和 Qwen3-30B-A3B 等模型。这些模型在代码、数学、通用能力等基准测试中表现出极具竞争力的结果，并已在 Hugging Face、ModelScope 和 Kaggle 等平台上开放使用。Qwen3 模型支持两种思考模式，具有灵活性，能够根据具体任务控制模型进行“思考”的程度。此外，Qwen3 模型支持 119 种语言和方言，为国际应用开辟了新的可能性。

## 4.2 多轮对话（ChatEngine）

In [32]:
# 创建聊天引擎
chat_engine = index.as_chat_engine()

# 第一个问题
response1 = chat_engine.chat("我的第一个问题是什么?")
print(response1)

# 跟进问题（保持上下文）
response2 = chat_engine.chat("还有其他相关信息吗?")
print(response2)

# 流式输出
streaming_chat_engine = index.as_chat_engine()
streaming_response = streaming_chat_engine.stream_chat("我想了解更多信息")
for token in streaming_response.response_gen:
    print(token, end="", flush=True)

您的第一个问题是关于 Qwen3 及其主要特点和功能的。
Qwen3 是通往通用人工智能（AGI）和超级人工智能（ASI）的重要里程碑。它通过扩大预训练和强化学习的规模实现了更高水平的智能。Qwen3 无缝集成了思考和非思考模式，使用户能够灵活控制他们的思考预算。此外，它支持多种语言，使全球用户受益。展望未来，计划从多个维度增强模型，包括优化模型架构和训练方法，以实现扩展数据规模、增加模型大小、延长上下文长度、拓展模态以及利用环境反馈推进长期推理的关键目标。这一转变被视为从专注于训练模型的时代转向专注于训练代理的时代，预计下一代迭代将为工作和生活带来有意义的进展。
请提供更具体的信息，这样我才能帮助您更好。您想了解什么样的信息？

## 5. 定制模型与Prompt

LlamaIndex提供了灵活的底层接口，允许开发者定制语言模型和提示词模板。

### 5.1 Prompt模板

使用PromptTemplate创建简单的提示词模板：

In [33]:
from llama_index.core import PromptTemplate

# 简单模板
prompt = PromptTemplate("写一个关于{topic}的{length}字短文")
formatted_prompt = prompt.format(topic="人工智能", length="500")
print(formatted_prompt)

写一个关于人工智能的500字短文


使用ChatPromptTemplate创建多轮对话模板：

In [34]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# 创建对话模板
chat_messages = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="你是一个专业的{role}，擅长解答{domain}问题。"
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "请根据以下上下文回答我的问题：\n"
            "{context}\n\n"
            "问题：{question}"
        )
    )
]

chat_template = ChatPromptTemplate(chat_messages)
formatted_chat = chat_template.format(
    role="人工智能助手",
    domain="技术",
    context="LlamaIndex是一个数据框架，连接LLM与外部数据。",
    question="LlamaIndex有什么用途?"
)

print(formatted_chat)

system: 你是一个专业的人工智能助手，擅长解答技术问题。
user: 请根据以下上下文回答我的问题：
LlamaIndex是一个数据框架，连接LLM与外部数据。

问题：LlamaIndex有什么用途?
assistant: 


### 5.2 使用不同的语言模型

LlamaIndex支持多种语言模型，以下是一些例子：

In [41]:
%pip install llama-index-llms-dashscope

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 187.9 kB/s eta 0:00:07
    --------------------------------------- 0.0/1.3 MB 187.9 kB/s eta 0:00:07
   - -------------------------------------- 0.0/1.3 MB 151.3 kB/s eta 0:00:09
   - -------------------------------------- 0.0/1.3 MB 151.3 kB/s eta 0:00:09
   - -------------------------------------- 0.0/1.3 MB 151.3 kB/s eta 0:00:09
   - -------------------------------------- 0.0/1.3 MB 151.3 kB/s eta 0:00:09
   - -------------------------------------- 0.0/1.3 MB 151.3 kB/s eta 0:00:09
   - -------------------------------------- 0.0/1.3 MB 151.3 kB/s eta 0:00:09
   -- -------------------------

In [43]:
# OpenAI模型
from llama_index.llms.openai import OpenAI
llm_openai = OpenAI(model="gpt-4o", temperature=0)

# 使用语言模型生成文本
response = llm_openai.complete("列出5个使用LlamaIndex的场景")
print(response.text)

# 使用本地模型或其他API提供商
# 以DeepSeek为例
from llama_index.llms.deepseek import DeepSeek
from llama_index.llms.dashscope import DashScope

# DeepSeek模型
llm_deepseek = DeepSeek(
    api_key=os.environ["DEEPSEEK_API_KEY"],
    model="deepseek-chat",
    temperature=0.7
)

# 阿里通义千问模型
llm_qwen = DashScope(
    api_key=os.environ["DASHSCOPE_API_KEY"],
    model="qwen-turbo",
    temperature=0.7
)

print("+++++++++++++++++++++++++++++++++++++++++++++++")

response = llm_deepseek.complete("简单介绍下什么是检索增强生成?")
print(response.text)

print("+++++++++++++++++++++++++++++++++++++++++++++++")

response = llm_qwen.complete("简单介绍下什么是检索增强生成?")
print(response.text)

LlamaIndex 是一个强大的工具，可以用于多种场景，特别是在数据索引和搜索方面。以下是五个使用 LlamaIndex 的场景：

1. **文档管理系统**：在企业或组织中，LlamaIndex 可以用于创建一个高效的文档管理系统。通过对大量文档进行索引，用户可以快速搜索和检索所需的信息，提高工作效率。

2. **电子商务网站搜索**：在电子商务平台上，LlamaIndex 可以帮助实现快速、准确的产品搜索功能。通过对产品描述、用户评论和其他相关数据进行索引，用户可以更容易地找到他们感兴趣的产品。

3. **学术研究数据库**：研究人员可以使用 LlamaIndex 来索引和搜索学术论文、研究报告和其他相关文献。这有助于快速查找相关研究成果，支持文献综述和研究分析。

4. **客户支持系统**：在客户支持中心，LlamaIndex 可以用于索引常见问题解答（FAQ）、支持文档和历史客户交互记录，从而帮助客服人员快速找到解决方案，提高客户满意度。

5. **内容推荐系统**：LlamaIndex 可以用于内容推荐引擎，通过对用户行为数据和内容特征进行索引，帮助提供个性化的内容推荐，提高用户参与度和留存率。

这些场景展示了 LlamaIndex 在不同领域的广泛应用潜力，帮助组织和个人更高效地管理和利用信息。
+++++++++++++++++++++++++++++++++++++++++++++++
检索增强生成（Retrieval-Augmented Generation, RAG）是一种结合信息检索与文本生成的技术，旨在提升生成式模型（如大语言模型）的准确性和事实性。其核心思想是通过实时检索外部知识库，为生成过程提供相关参考信息，从而减少模型因依赖内部参数化知识而产生的错误或幻觉（Hallucination）。

### **核心流程**
1. **检索（Retrieval）**  
   当用户输入查询（Query）时，系统从外部数据库（如维基百科、专业文档等）检索与问题最相关的文档或段落。通常使用向量检索技术（如稠密检索Dense Retrieval），将查询和文档编码为向量，通过相似度匹配找出相关内容。

2. **增强（Augmentation）**  
   检索到的文档片段与用户查询一起拼接，作为生成模型的输入上下文。例如：  
  

## 6. 构建完整的RAG系统

最后，让我们综合前面所学的知识，构建一个完整的RAG系统：

In [44]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 加载环境变量
load_dotenv()

# 全局设置
Settings.llm = OpenAI(model="gpt-4o", temperature=0.2)
Settings.chunk_size = 1024
Settings.chunk_overlap = 200

# 1. 加载文档
documents = SimpleDirectoryReader("./data/llamaindex_data/").load_data()
print(f"加载了 {len(documents)} 个文档")

# 2. 切分文档
node_parser = TokenTextSplitter(
    chunk_size=Settings.chunk_size,
    chunk_overlap=Settings.chunk_overlap
)
nodes = node_parser.get_nodes_from_documents(documents)
print(f"生成了 {len(nodes)} 个文本节点")

# 3. 创建索引
index = VectorStoreIndex(nodes)

# 4. 保存索引（可选）
index.storage_context.persist("./storage")

# 5. 创建查询引擎
query_engine = index.as_query_engine(
    similarity_top_k=3,
    streaming=True
)

# 6. 创建对话引擎
chat_engine = index.as_chat_engine(
    similarity_top_k=3,
    streaming=True
)

# 7. 与系统交互
while True:
    query = input("\n请输入问题(输入'exit'退出): ")
    if query.lower() == "exit":
        break
    
    print("\n正在生成回答...\n")
    response = query_engine.query(query)
    response.print_response_stream()

加载了 4 个文档
生成了 4 个文本节点

正在生成回答...

DeepSeek-V3 excels in a variety of tasks, including educational benchmarks, factuality benchmarks, math-related tasks, and coding-related tasks. It performs exceptionally well on educational benchmarks like MMLU, MMLU-Pro, and GPQA, and demonstrates strong capabilities in factual knowledge, particularly in Chinese. Additionally, it achieves state-of-the-art performance on math-related benchmarks and is a top performer in coding competitions, making it highly competitive across diverse technical benchmarks.

LlamaIndex的生态系统正在快速发展，除了本文介绍的基础功能外，它还支持更多高级特性，如代理系统、知识图谱、查询引擎评估等。随着你的深入学习，这些功能将帮助你构建更强大、更智能的应用程序。
希望本文能帮助你快速上手LlamaIndex，开始构建自己的RAG应用！